In [3]:
from nltk.corpus import gutenberg, stopwords
import nltk

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.utils import np_utils

from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.word2vec import Word2Vec

import string
import numpy as np
import pandas as pd
import re
import os
# set seed for reproducibility
np.random.seed(0)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
all_files = os.listdir("book/")

In [5]:
arr = []
one_book = []
content = ''
path = 'book/'
for b in all_files[0:80]:
    one_book = []
    path = 'book/'
    one_book.append(b.split('___')[0])
    one_book.append(b)
    
    path += b
    with open(path, 'r') as myfile:
        content = myfile.read().replace('\n', ' ')
    one_book.append(content)
    arr.append(one_book)

In [6]:
columns = ['author', 'file_name', 'text']
df = pd.DataFrame(arr, columns=columns)

In [7]:
df.shape

(80, 3)

In [8]:
df.head()

,author,file_name,text
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,My dear father:-- Your letter of the 7th was ...
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,Fellow citizens of the United States: in comp...
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,"Four score and seven years ago, our fathers br..."
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",INTRODUCTION The facts of Lincoln's early lif...
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,Fellow countrymen: At this second appearing t...


In [9]:
stop_words = set(stopwords.words('english'))

In [10]:
def get_data_of_file(row):
    file_name = row.file_name
    text = ''
    
    words = row.text.split()
    for r in words:
        if not r in stop_words:
            text += ' ' + r
    
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    row['text'] = text
    return row
df = df.apply(get_data_of_file, axis=1)

In [11]:
df.head()

,author,file_name,text
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,My dear father : - - Your letter 7th received...
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,Fellow citizens United States : compliance cu...
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,Four score seven years ago fathers brought fo...
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",INTRODUCTION The facts Lincoln early life bes...
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,Fellow countrymen : At second appearing take ...


In [12]:
df.shape

(80, 3)

### Word2Vec

In [13]:
def get_good_tokens(sentence):
    replaced_punctation = list(map(lambda token: re.sub('[^0-9A-Za-z!?]+', '', token), sentence))
    removed_punctation = list(filter(lambda token: token, replaced_punctation))
    return removed_punctation

In [14]:
def w2v_preprocessing(df):
    """ All the preprocessing steps for word2vec are done in this function.
    All mutations are done on the dataframe itself. So this function returns
    nothing.
    """
    df['text'] = df.text.str.lower()
    df['document_sentences'] = df.text.str.split('.')  # split texts into individual sentences
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         df.document_sentences))  # tokenize sentences
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(map(get_good_tokens, sentences)),
                                         df.tokenized_sentences))  # remove unwanted characters
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(filter(lambda lst: lst, sentences)),
                                         df.tokenized_sentences))  # remove empty lists

w2v_preprocessing(df)

In [15]:
df.head()

,author,file_name,text,document_sentences,tokenized_sentences
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,my dear father : - - your letter 7th received...,[ my dear father : - - your letter 7th receive...,"[[my, dear, father, your, letter, 7th, receive..."
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,fellow citizens united states : compliance cu...,[ fellow citizens united states : compliance c...,"[[fellow, citizens, united, states, compliance..."
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,four score seven years ago fathers brought fo...,[ four score seven years ago fathers brought f...,"[[four, score, seven, years, ago, fathers, bro..."
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",introduction the facts lincoln early life bes...,[ introduction the facts lincoln early life be...,"[[introduction, the, facts, lincoln, early, li..."
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,fellow countrymen : at second appearing take ...,[ fellow countrymen : at second appearing take...,"[[fellow, countrymen, at, second, appearing, t..."


In [16]:
sentences = []
for sentence_group in df.tokenized_sentences:
    sentences.extend(sentence_group)

print("Number of sentences: {}.".format(len(sentences)))
print("Number of texts: {}.".format(len(df)))

Number of sentences: 80.
Number of texts: 80.


In [17]:
# Set values for various parameters
num_features = 200    # Word vector dimensionality
min_word_count = 3    # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model
W2Vmodel = Word2Vec(sentences=sentences,
                    sg=1,
                    hs=0,
                    workers=num_workers,
                    size=num_features,
                    min_count=min_word_count,
                    window=context,
                    sample=downsampling,
                    negative=5,
                    iter=6)

In [18]:
# W2Vmodel['sentences']

In [19]:
def get_w2v_features(w2v_model, sentence_group):
    """ Transform a sentence_group (containing multiple lists
    of words) into a feature vector. It averages out all the
    word vectors of the sentence_group.
    """
    words = np.concatenate(sentence_group)  # words in text
    index2word_set = set(w2v_model.wv.vocab.keys())  # words known to model
    
    featureVec = np.zeros(w2v_model.vector_size, dtype="float32")
    
    # Initialize a counter for number of words in a review
    nwords = 0
    # Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec = np.add(featureVec, w2v_model[word])
            nwords += 1.

    # Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

df['w2v_features'] = list(map(lambda sen_group:
                                      get_w2v_features(W2Vmodel, sen_group),
                                      df.tokenized_sentences))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [20]:
df.head()

,author,file_name,text,document_sentences,tokenized_sentences,w2v_features
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,my dear father : - - your letter 7th received...,[ my dear father : - - your letter 7th receive...,"[[my, dear, father, your, letter, 7th, receive...","[0.16781017, -0.09903394, -0.28047475, -0.2280..."
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,fellow citizens united states : compliance cu...,[ fellow citizens united states : compliance c...,"[[fellow, citizens, united, states, compliance...","[0.085581504, -0.09073201, -0.24187273, -0.248..."
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,four score seven years ago fathers brought fo...,[ four score seven years ago fathers brought f...,"[[four, score, seven, years, ago, fathers, bro...","[0.13051939, -0.110352755, -0.2540632, -0.2566..."
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",introduction the facts lincoln early life bes...,[ introduction the facts lincoln early life be...,"[[introduction, the, facts, lincoln, early, li...","[0.110757194, -0.08897842, -0.24979639, -0.254..."
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,fellow countrymen : at second appearing take ...,[ fellow countrymen : at second appearing take...,"[[fellow, countrymen, at, second, appearing, t...","[0.1265721, -0.09276668, -0.22800617, -0.26777..."


In [21]:
le = LabelEncoder()
df['author_LabelEncoded'] = le.fit_transform(df.author)

In [22]:
df.head()

,author,file_name,text,document_sentences,tokenized_sentences,w2v_features,author_LabelEncoded
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,my dear father : - - your letter 7th received...,[ my dear father : - - your letter 7th receive...,"[[my, dear, father, your, letter, 7th, receive...","[0.16781017, -0.09903394, -0.28047475, -0.2280...",0
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,fellow citizens united states : compliance cu...,[ fellow citizens united states : compliance c...,"[[fellow, citizens, united, states, compliance...","[0.085581504, -0.09073201, -0.24187273, -0.248...",0
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,four score seven years ago fathers brought fo...,[ four score seven years ago fathers brought f...,"[[four, score, seven, years, ago, fathers, bro...","[0.13051939, -0.110352755, -0.2540632, -0.2566...",0
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",introduction the facts lincoln early life bes...,[ introduction the facts lincoln early life be...,"[[introduction, the, facts, lincoln, early, li...","[0.110757194, -0.08897842, -0.24979639, -0.254...",0
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,fellow countrymen : at second appearing take ...,[ fellow countrymen : at second appearing take...,"[[fellow, countrymen, at, second, appearing, t...","[0.1265721, -0.09276668, -0.22800617, -0.26777...",0


In [23]:
X_train, X_test, y_train, y_test = train_test_split(df.w2v_features, df.author_LabelEncoded, random_state = 0)

In [24]:
X_train_w2v = np.array(list(map(np.array, X_train)))
X_test_w2v = np.array(list(map(np.array, X_test)))

In [25]:
clf = SVC(kernel='linear').fit(X_train_w2v, y_train)

In [26]:
y_score = clf.predict(X_test_w2v)

In [27]:
n_right = 0
for i in range(len(y_score)):
    if y_score.tolist()[i] == y_test.tolist()[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 50.00%


In [28]:
print(classification_report(y_test, y_score))

             precision    recall  f1-score   support

          0       0.75      1.00      0.86         3
          3       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         2
          5       1.00      0.50      0.67         2
          6       0.38      0.60      0.46         5
          7       0.43      1.00      0.60         3
          8       0.00      0.00      0.00         4

avg / total       0.37      0.50      0.40        20



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=42)

In [30]:
rfc.fit(X_train_w2v, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [31]:
y_score = clf.predict(X_test_w2v)

In [32]:
print(classification_report(y_test, y_score))

             precision    recall  f1-score   support

          0       0.75      1.00      0.86         3
          3       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         2
          5       1.00      0.50      0.67         2
          6       0.38      0.60      0.46         5
          7       0.43      1.00      0.60         3
          8       0.00      0.00      0.00         4

avg / total       0.37      0.50      0.40        20



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
df.head()

,author,file_name,text,document_sentences,tokenized_sentences,w2v_features,author_LabelEncoded
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,my dear father : - - your letter 7th received...,[ my dear father : - - your letter 7th receive...,"[[my, dear, father, your, letter, 7th, receive...","[0.16781017, -0.09903394, -0.28047475, -0.2280...",0
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,fellow citizens united states : compliance cu...,[ fellow citizens united states : compliance c...,"[[fellow, citizens, united, states, compliance...","[0.085581504, -0.09073201, -0.24187273, -0.248...",0
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,four score seven years ago fathers brought fo...,[ four score seven years ago fathers brought f...,"[[four, score, seven, years, ago, fathers, bro...","[0.13051939, -0.110352755, -0.2540632, -0.2566...",0
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",introduction the facts lincoln early life bes...,[ introduction the facts lincoln early life be...,"[[introduction, the, facts, lincoln, early, li...","[0.110757194, -0.08897842, -0.24979639, -0.254...",0
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,fellow countrymen : at second appearing take ...,[ fellow countrymen : at second appearing take...,"[[fellow, countrymen, at, second, appearing, t...","[0.1265721, -0.09276668, -0.22800617, -0.26777...",0


### TF-IDF

In [35]:
vectorizer = TfidfVectorizer(sublinear_tf=True, use_idf =True, lowercase=True, strip_accents='ascii', stop_words = 'english')

In [36]:
XText_tfidf = vectorizer.fit_transform(df['text'])

In [37]:
XText_tfidf.shape

(80, 70433)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(XText_tfidf, df.author_LabelEncoded, random_state = 0)

In [39]:
clf = MultinomialNB().fit(X_train, y_train)

In [40]:
y_score = clf.predict(X_test)

In [41]:
n_right = 0
for i in range(len(y_score)):
    if y_score.tolist()[i] == y_test.tolist()[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 50.00%


In [42]:
print(classification_report(y_test, y_score))

             precision    recall  f1-score   support

          0       0.60      1.00      0.75         3
          3       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         2
          5       0.00      0.00      0.00         2
          6       0.40      0.80      0.53         5
          7       0.60      1.00      0.75         3
          8       0.00      0.00      0.00         4

avg / total       0.28      0.50      0.36        20



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [43]:
clf = SVC(kernel='linear').fit(X_train, y_train)

In [44]:
y_score = clf.predict(X_test)

In [45]:
n_right = 0
for i in range(len(y_score)):
    if y_score.tolist()[i] == y_test.tolist()[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 95.00%


In [46]:
print(classification_report(y_test, y_score))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          3       0.00      0.00      0.00         1
          4       1.00      1.00      1.00         2
          5       1.00      1.00      1.00         2
          6       1.00      1.00      1.00         5
          7       0.75      1.00      0.86         3
          8       1.00      1.00      1.00         4

avg / total       0.91      0.95      0.93        20



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [47]:
df.head()

,author,file_name,text,document_sentences,tokenized_sentences,w2v_features,author_LabelEncoded
0,Abraham Lincoln,Abraham Lincoln___Lincoln Letters.txt,my dear father : - - your letter 7th received...,[ my dear father : - - your letter 7th receive...,"[[my, dear, father, your, letter, 7th, receive...","[0.16781017, -0.09903394, -0.28047475, -0.2280...",0
1,Abraham Lincoln,Abraham Lincoln___Lincoln's First Inaugural Ad...,fellow citizens united states : compliance cu...,[ fellow citizens united states : compliance c...,"[[fellow, citizens, united, states, compliance...","[0.085581504, -0.09073201, -0.24187273, -0.248...",0
2,Abraham Lincoln,Abraham Lincoln___Lincoln's Gettysburg Address...,four score seven years ago fathers brought fo...,[ four score seven years ago fathers brought f...,"[[four, score, seven, years, ago, fathers, bro...","[0.13051939, -0.110352755, -0.2540632, -0.2566...",0
3,Abraham Lincoln,"Abraham Lincoln___Lincoln's Inaugurals, Addres...",introduction the facts lincoln early life bes...,[ introduction the facts lincoln early life be...,"[[introduction, the, facts, lincoln, early, li...","[0.110757194, -0.08897842, -0.24979639, -0.254...",0
4,Abraham Lincoln,Abraham Lincoln___Lincoln's Second Inaugural A...,fellow countrymen : at second appearing take ...,[ fellow countrymen : at second appearing take...,"[[fellow, countrymen, at, second, appearing, t...","[0.1265721, -0.09276668, -0.22800617, -0.26777...",0


### Bag Of Word

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
count_vectorizer = CountVectorizer(stop_words='english')
XText_CountVectorizer = count_vectorizer.fit_transform(df['text'])

In [50]:
XText_CountVectorizer.shape[1]

70433

In [51]:
X_train, X_test, y_train, y_test = train_test_split(XText_CountVectorizer, df.author_LabelEncoded, random_state = 0)

In [52]:
clf = MultinomialNB().fit(X_train, y_train)

In [53]:
y_score = clf.predict(X_test)

In [54]:
n_right = 0
for i in range(len(y_score)):
    if y_score.tolist()[i] == y_test.tolist()[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 95.00%


In [55]:
print(classification_report(y_test, y_score))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          3       0.00      0.00      0.00         1
          4       1.00      1.00      1.00         2
          5       1.00      1.00      1.00         2
          6       0.83      1.00      0.91         5
          7       1.00      1.00      1.00         3
          8       1.00      1.00      1.00         4

avg / total       0.91      0.95      0.93        20



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [56]:
clf = SVC(kernel='linear').fit(X_train, y_train)

In [57]:
y_score = clf.predict(X_test)

In [58]:
y_score.shape

(20,)

In [59]:
n_right = 0
for i in range(len(y_score)):
    if y_score.tolist()[i] == y_test.tolist()[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 85.00%


In [60]:
print(classification_report(y_test.tolist(), y_score.tolist()))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          1       0.00      0.00      0.00         0
          3       0.00      0.00      0.00         1
          4       1.00      1.00      1.00         2
          5       1.00      1.00      1.00         2
          6       0.71      1.00      0.83         5
          7       1.00      1.00      1.00         3
          8       1.00      0.50      0.67         4

avg / total       0.88      0.85      0.84        20



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Word Embedding

In [61]:
### Create sequence
vocabulary_size = XText_CountVectorizer.shape[1]
num_of_author = len(df['author_LabelEncoded'].unique())

tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=5000)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(data, np_utils.to_categorical(df['author_LabelEncoded']), random_state = 0)

In [63]:
model = Sequential()
model.add(Embedding(vocabulary_size, 256, input_length = data.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_of_author, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=5)

Epoch 1/10
60/60 [==============================] - 79s 1s/step - loss: 2.1942 - acc: 0.1500
Epoch 2/10
60/60 [==============================] - 72s 1s/step - loss: 2.0817 - acc: 0.3500
Epoch 3/10
60/60 [==============================] - 74s 1s/step - loss: 1.8706 - acc: 0.3500
Epoch 4/10
60/60 [==============================] - 73s 1s/step - loss: 1.6823 - acc: 0.5667
Epoch 5/10
60/60 [==============================] - 72s 1s/step - loss: 1.2565 - acc: 0.8500
Epoch 6/10
60/60 [==============================] - 80s 1s/step - loss: 0.7596 - acc: 0.9333
Epoch 7/10
60/60 [==============================] - 80s 1s/step - loss: 0.4721 - acc: 0.9667
Epoch 8/10
60/60 [==============================] - 84s 1s/step - loss: 0.3384 - acc: 0.9500
Epoch 9/10
60/60 [==============================] - 84s 1s/step - loss: 0.2462 - acc: 0.9500
Epoch 10/10
60/60 [==============================] - 82s 1s/step - loss: 0.1773 - acc: 0.9667


In [64]:
y_score = model.predict(X_test)
y_score = [[1 if i == max(sc) else 0 for i in sc] for sc in y_score]
n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == np.array(y_test)[i][j] for j in range(len(y_score[i]))):
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 15.00%
